# Tutorial: Working with this stactools subpackage

Stactools ([docs](https://stactools.readthedocs.io/en/latest/), [source](https://github.com/stac-utils/stactools)) is a command line tool and library for working with [STAC](https://stacspec.org/), based on [PySTAC](https://github.com/stac-utils/pystac). [Stactools packages](https://github.com/stactools-packages) are add-ons for stactools that provide STAC functionality for specific datasets, such as [Sentinel 2](https://github.com/stactools-packages/sentinel2) and [Landsat](https://github.com/stactools-packages/landsat). 

Stactools and its packages can be accessed from the CLI or from within normal Python code. This notebook provides examples of both.

## 1. Installing stactools

To use a package, first install `stactools` then the package. `stactools` can be installed with `pip`.

In [1]:
%pip install stactools

You should consider upgrading via the '/home/james/Projects/Community/STAC/stactools/.venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


Check that the `stac` CLI tool is installed

In [2]:
!stac

Usage: stac [OPTIONS] COMMAND [ARGS]...

Options:
  -v, --verbose  Use verbose mode
  -q, --quiet    Use quiet mode (no output)
  --help         Show this message and exit.

Commands:
  copy         Copy a STAC Catalog
  describe     Prints out a list of all catalogs, collections and items in
               this STAC.

  info         Display info about a static STAC catalog.
  jrc-gsw      Commands for working with JRC-GSW data.
  layout       Reformat the layout of a STAC based on templating.
  merge        Merge items from one STAC into another.
  move-assets  Move or copy assets in a STAC to the Item locations.
  version      Display version info.


Notice the Commands available. In the next step we will add a dataset to work with

## 2. Installing a stactools package

Here we'll use the subpackage that this notebook resides within, but feel free to change the `PACKAGE` variable to any one of the subpackage repo names in the [stactools-subpackages](https://github.com/stactools-packages). Other examples include `sentinel2`, `planet`, and `landsat`.

In [3]:
# Alter this to match your package
PACKAGE = "nrcan-landcover"

In [4]:
%pip install stactools-{PACKAGE}

  Using cached stactools_nrcan_landcover-0.2.3-py3-none-any.whl (8.4 kB)
You should consider upgrading via the '/home/james/Projects/Community/STAC/stactools/.venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


Notice the addition of the subpackage command in stactools now.

In [5]:
!stac

Usage: stac [OPTIONS] COMMAND [ARGS]...

Options:
  -v, --verbose  Use verbose mode
  -q, --quiet    Use quiet mode (no output)
  --help         Show this message and exit.

Commands:
  copy            Copy a STAC Catalog
  describe        Prints out a list of all catalogs, collections and items in
                  this STAC.

  info            Display info about a static STAC catalog.
  jrc-gsw         Commands for working with JRC-GSW data.
  layout          Reformat the layout of a STAC based on templating.
  merge           Merge items from one STAC into another.
  move-assets     Move or copy assets in a STAC to the Item locations.
  nrcanlandcover  Commands for working with Natural Resources Canada Land
                  Cover data

  version         Display version info.


## 3. Using the stactools subpackage

You can now explore the STAC package commands to ingest and describe the data

In [6]:
# Alter this to match your package
COMMAND = "create-item"

In [7]:
!stac {PACKAGE.replace("-", "")} --help

Usage: stac nrcanlandcover [OPTIONS] COMMAND [ARGS]...

Options:
  --help  Show this message and exit.

Commands:
  create-cog         Transform Geotiff to Cloud-Optimized Geotiff.
  create-collection  Creates a STAC collection from NRCan Landcover metadata
  create-item        Create a STAC item using JSONLD metadata and a COG


And more specific help with an individual command.

In [8]:
!stac {PACKAGE.replace("-", "")} {COMMAND} --help

Usage: stac nrcanlandcover create-item [OPTIONS]

  Generate a STAC item using the metadata, with an asset url as provided.

  Args:     destination (str): Local directory to save the STAC Item json
  cog (str): location of a COG asset for the item     metadata (str): url
  containing the NRCAN Landcover JSONLD metadata

Options:
  -d, --destination TEXT  The output directory for the STAC json  [required]
  -c, --cog TEXT          COG href  [required]
  -m, --metadata TEXT     The url to the metadata description.
  --help                  Show this message and exit.


So far we've used Jupyter Notebooks' IPython [line magic](https://ipython.readthedocs.io/en/stable/interactive/magics.html) to work with stactools packages on the command line, but it's also possible to use them within Python scripts.

In [9]:
# Alter these with examples to match your package
from stactools.nrcan_landcover import stac, cog

stac.create_item()
stac.create_collection()
cog.create_cog()

TypeError: create_item() missing 2 required positional arguments: 'metadata' and 'metadata_url'